# Canonical Weak Tournaments and Canonical Weighted Tournaments

In [23]:
from itertools import product, combinations, permutations
import pickle
import networkx as nx # for graphs
from tqdm.notebook import tqdm
from scipy.special import comb
from copy import deepcopy


In [24]:
# helper functions

def is_maj_preferred(wt, c1, c2): 
    """True if c1 is majority preferred to c2"""
    return wt.has_edge(c1, c2)

def is_tournament(wt):
    """test if a weak tournament is a tournament"""
    candidates = wt.nodes
    is_t = True
    for c1 in candidates: 
        for c2 in candidates: 
            if c1 != c2 and not is_maj_preferred(wt,c1,c2) and not is_maj_preferred(wt,c2,c1):
                is_t = False
    return is_t

def findsubsets(s, n):
    """all subsets of the list s of size n""" 
    return [set(i) for i in combinations(s, n)] 

def ways(weight_domain, num):
    return [x for x in combinations(weight_domain, int(num))]

# 1. Canonical weak tournaments

Our first step is to create the canonical (weak) tournaments referenced in Definition A.4. We construct all the canonical (weak) tournaments of size 2, 3, 4, 5 and 6.  

Tournaments are represented by [networkx Digraph](https://networkx.org/documentation/stable/tutorial.html#directed-graphs) objects.   

In [28]:
# Only consider tournaments up to six candidates
numbers_of_candidates = [2, 3]#, 4, 5, 6]

# Warning: Constructing the canonical graphs for size 6 takes a very long time!

In [32]:

#  nx.graph_atlas_g() is a list of all the undirected graphs with up to 7 nodes
# See https://networkx.org/documentation/networkx-1.10/reference/generated/networkx.generators.atlas.graph_atlas_g.html
undirected_graphs_size_2 = nx.graph_atlas_g()[2:4] # undirected graphs with 2 nodes
undirected_graphs_size_3 = nx.graph_atlas_g()[4:8] # undirected graphs with 3 nodes
undirected_graphs_size_4 = nx.graph_atlas_g()[8:19] # undirected graphs with 4 nodes
undirected_graphs_size_5 = nx.graph_atlas_g()[19:53] # undirected graphs with 5 nodes
undirected_graphs_size_6 = nx.graph_atlas_g()[53:209] # undirected graphs with 6 nodes

undirected_graphs = {2: undirected_graphs_size_2,
                     3: undirected_graphs_size_3, 
                     4: undirected_graphs_size_4,
                     5: undirected_graphs_size_5, 
                     6: undirected_graphs_size_6}

canonical_weak_tournaments = {nc: list() for nc in numbers_of_candidates}

for nc in numbers_of_candidates: 

    for g in tqdm(undirected_graphs[nc]):

        num_edges = len(g.edges)

        # for n=0 to the number of edges
        for n in range(num_edges + 1): 

            # find all the subsets of edges of size n
            forward_edges = findsubsets(g.edges, n)

            # for each set of edges
            for f_edges in tqdm(forward_edges, leave=False): 

                # f_edges is the set of forward edges, the other edges in g point in the opposite direction
                d_edges = list(f_edges) + [(e[1], e[0]) for e in g.edges if e not in f_edges]

                # create a directed graph with the nodes from g and directed edges in d_edges
                d_graph = nx.DiGraph()
                d_graph.add_nodes_from(g.nodes)
                d_graph.add_edges_from(d_edges)

                # check if we have already seen a graph isomorphic to d_graph
                if all([not nx.is_isomorphic(d_graph,_t) for _t in canonical_weak_tournaments[nc]]): 
                    canonical_weak_tournaments[nc].append(d_graph)
        
        # uncomment to save the graphs
        #pickle.dump(canonical_weak_tournaments[nc], open(f"weak_tourns_{nc}.pkl", "wb"))

print("Done loading canonical weak tournaments.\n")

for nc in numbers_of_candidates: 
    print(f"There are {len(canonical_weak_tournaments[nc])} canonical weak tournaments for {nc} candidates")

    
# Set aside the tournaments
canonical_tournaments = {n_cands: list() for n_cands in canonical_weak_tournaments.keys()}
for n_cands in canonical_weak_tournaments.keys():
    for t in canonical_weak_tournaments[n_cands]:
        if is_tournament(t):
            canonical_tournaments[n_cands].append(t)
    # uncomment to save the graphs
    #pickle.dump(canonical_tournaments[n_cands], open(f"tourns_{n_cands}.pkl", "wb"))

print(canonical_tournaments)

print("\n\nDone loading canonical tournaments.\n")

for nc in numbers_of_candidates: 
    print(f"There are {len(canonical_tournaments[nc])} canonical tournaments for {nc} candidates")
    
    


Done loading canonical weak tournaments.

There are 2 canonical weak tournaments for 2 candidates
There are 7 canonical weak tournaments for 3 candidates
2
3
{2: [<networkx.classes.digraph.DiGraph object at 0x7ff2403df220>], 3: [<networkx.classes.digraph.DiGraph object at 0x7ff1e01d0550>, <networkx.classes.digraph.DiGraph object at 0x7ff1e01d0280>]}


Done loading canonical tournaments.

There are 1 canonical tournaments for 2 candidates
There are 2 canonical tournaments for 3 candidates


# 2. Canonical weighted tournaments


In [33]:

numbers_of_candidates = [2, 3, 4]

# name of networkx edge attribute storing each edge's weight
edge_attr = 'weight'

# possible weights on edges
weight_domain = [2, 4, 6, 8, 10, 12]

def edge_compare_helper(dict1, dict2, edge_attr):
    return dict1[edge_attr] == dict2[edge_attr]

edge_compare = (lambda edge_attr: lambda dict1, dict2: edge_compare_helper(dict1, dict2, edge_attr))(edge_attr)

In [34]:
# Paths to pickled weak tournaments
canonical_tournament_paths = {
    2: 'tourns/tourns_2.pkl',
    3: 'tourns/tourns_3.pkl',
    4: 'tourns/tourns_4.pkl',
}

# Load all the canonical weak tournaments
canonical_tournaments = {nc: pickle.load(open(canonical_tournament_paths[nc], 'rb')) \
                         for nc in numbers_of_candidates}

print("Done loading canonical tournaments.\n")

for nc in numbers_of_candidates: 
    print(f"There are {len(canonical_tournaments[nc])} canonical tournaments for {nc} candidates")


can_graphs = canonical_tournaments

canonical_weighted_tournaments = {}

for nc, graphs in can_graphs.items():
    weighted_tournaments_nc = []

    # all combinations of unique edge weights assignable to edges
    weightings = ways(weight_domain, comb(nc, 2))
    
    for graph in graphs:
        
        for weighting in weightings:
            weighted_tournaments_nc_margin_i = []
            
            # all permutations of the assignable edge weights on the edges
            perms = list(permutations(weighting))
            
            for perm in perms:                    
                g1 = deepcopy(graph)
                
                # assign particular permutation of edge weights to the graph
                for i, e in enumerate(g1.edges):
                    g1.edges[e][edge_attr] = perm[i]

                # if this permutation is isomorphic to one already seen, discard it;
                # else add it to the running list of canonical weighted tournaments
                if any([nx.algorithms.isomorphism.is_isomorphic(g1, g2, edge_match=edge_compare) \
                        for g2 in weighted_tournaments_nc_margin_i]):
                    continue
                else:
                    weighted_tournaments_nc_margin_i.append(g1)

            weighted_tournaments_nc += weighted_tournaments_nc_margin_i
                
    canonical_weighted_tournaments[nc] = weighted_tournaments_nc
    # uncomment to save the graphs
    #pickle.dump(canonical_weighted_tournaments[nc], open(f"weighted_tourns_{nc}.pkl", "wb"))
        
for nc in numbers_of_candidates:
    print(f"There are {len(canonical_weighted_tournaments[nc])} canonical weighted tournaments for {nc} candidates")


Done loading canonical tournaments.

There are 1 canonical tournaments for 2 candidates
There are 2 canonical tournaments for 3 candidates
There are 4 canonical tournaments for 4 candidates
There are 1920 canonical weighted tournaments for 4 candidates
